<a href="https://colab.research.google.com/github/Jeevesh8/AutoRegressive-MLM/blob/main/AutoRegressive_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!git clone https://github.com/Jeevesh8/AutoRegressive-MLM
%cd AutoRegressive-MLM
!pip install -r requirements.txt

In [ ]:
import jax
import jax.numpy as jnp
import haiku as hk
from haiku.data_structures import to_immutable_dict
import optax

import copy
import numpy as np
from functools import partial

In [ ]:
from src.DataLoaders.json import load_reddit_data
from src.Tokenizers.tree_tokenizer import Tree_Tokenizer
from src.model.transformer import TransformerFeaturizer, ExtendedEncoder
from src.optimizers.adam import get_adam_opt
from src.Tokenizers.masking_utils import mask_batch_mlm
from src.Tokenizers.utils import tree_to_batch, batch_to_tree, gather_batch_parents

## Setting Up Config

In [ ]:
config = {
          #Data Parameters
          'max_length' : 128, 
          'featurizer_batch_size' : 4,
          'mlm_batch_size' : 4,
          'data_files' : ['/content/drive/MyDrive/2SCL/Argumentation/first_batch_data/heldout_period_data.jsonlist'],

          #Model Parameters
          'intermediate_size' : 256,
          'n_heads' : 2,
          'n_layers' : 2,
          'hidden_size' : 128,
          'd_model' : 128,                                                      #same as hidden_size
          
          #Embeddings Parameters
          'embed_dropout_rate' : 0.1,
          
          #MHA parameters
          'attention_drop_rate' : 0.1,
          
          #MLP parameters
          'fully_connected_drop_rate' : 0.1,
          
          #Training Parameters
          'learning_rate' : 1e-5,
          'max_grad_norm' : 1.0,
          'l2' : 0.1,
          
          }


## Getting Data

In [ ]:
data_loader = load_reddit_data(config)

## Training Tokenizer


In [ ]:
def get_sentences():
    for tree in data_loader.tree_generator():
        yield tree['title'] + ' ' + tree['selftext']
        for id, comment in tree['comments'].items():
            yield comment['body']

In [ ]:
lm_tokeniser = Tree_Tokenizer(config)
lm_tokeniser.train_tokenizer(str_iter=get_sentences())

In [ ]:
print(lm_tokeniser.tokenizer.get_vocab())

{'holy': 8613, 'thou': 17718, 'Arm': 25244, 'represents': 5553, 'ridden': 16852, 'Chand': 23834, 'umping': 17062, 'ience': 2732, 'Univers': 10259, 'quarantined': 27105, 'insects': 12753, 'abil': 19125, 'youre': 11032, 'CBO': 27207, 'abstain': 11755, '7': 28, 'contracted': 23965, 'predates': 25232, 'numer': 14006, 'controversi': 5048, 'MK': 13134, 'isible': 9918, 'arresting': 19265, 'Democrats': 4867, 'outward': 14546, 'welfare': 3167, 'trill': 6985, 'average': 1871, 'prim': 2115, 'rants': 10436, 'confine': 15351, 'cbs': 29156, 'institutions': 5618, 'ani': 26496, 'bur': 1536, 'France': 6148, 'Thanks': 2471, 'unprotected': 15135, 'agans': 27396, 'igger': 2903, 'MB': 22092, 'ironically': 12909, ').[': 25143, 'unknown': 6187, 'Roosevelt': 17671, 'vian': 12957, 'replacements': 22891, 'presidential': 4906, 'associated': 4009, 'ensive': 1674, 'zero': 3212, 'Responding': 11643, 'FDA': 11812, 'Presumably': 14941, 'Goodell': 21611, 'sycamore': 17433, 'tyran': 6965, 'Moz': 20972, 'Topic': 17900, 

### Updating Config

In [ ]:
config['vocab_size'] = lm_tokeniser.tokenizer.get_vocab_size()

#Tokenization ids  
config['mask_id'] = lm_tokeniser.tokenizer.token_to_id("<mask>")
config['pad_id'] = lm_tokeniser.tokenizer.token_to_id("<pad>")
config['sos_id'] = lm_tokeniser.tokenizer.token_to_id("<s>")
config['eos_id'] = lm_tokeniser.tokenizer.token_to_id("</s>")
config['total_steps'] = len([0 for tree in data_loader.tree_generator()])
config = hk.data_structures.to_immutable_dict(config)

In [ ]:
print(config['total_steps'])

2263


## Purifying the Model Functions and Getting Parameters

In [ ]:
def featurizer(token_ids, training, config):
    features = TransformerFeaturizer(config)(token_ids, training=training)
    return features

def logits_fn(comment_embds, comment_mask, masked_token_ids, training, config):
    logits = ExtendedEncoder(config)(comment_embds, comment_mask, 
                                     masked_token_ids, training=training)
    return logits

key, subkey = jax.random.split( jax.random.PRNGKey(42) )
pure_logits_fn = hk.transform(logits_fn)
pure_featurizer_fn = hk.transform(featurizer)

comment_encoding = lm_tokeniser.batch_encode_plus(['sample sentence']*config['featurizer_batch_size'])
token_encoding = lm_tokeniser.batch_encode_plus(['sample sentence']*config['mlm_batch_size'])

token_ids = np.asarray(lm_tokeniser.get_token_ids(token_encoding), dtype=np.int16)
comment_ids = np.asarray(lm_tokeniser.get_token_ids(comment_encoding), dtype=np.int16)

masked_token_ids, original_batch = mask_batch_mlm(subkey, config, token_ids)

In [ ]:
key, subkey = jax.random.split(key)
featurizer_params = pure_featurizer_fn.init(subkey, comment_ids, True, config)

key, subkey = jax.random.split(key)
comment_embds = pure_featurizer_fn.apply(featurizer_params, subkey, comment_ids, True, config)

In [ ]:
print(comment_embds.shape)
print(jnp.tile(comment_embds, config['max_length']).reshape(config['mlm_batch_size'], config['max_length'], -1).shape)
print(comment_embds.dtype, masked_token_ids.dtype)

In [ ]:
key, subkey = jax.random.split(key)

comment_embds = jnp.tile(comment_embds, config['max_length']).reshape(config['mlm_batch_size'], config['max_length'], -1)
comment_mask = jnp.ones_like(comment_embds[:,:,0])

ExtendedEncoder_params = pure_logits_fn.init(subkey, comment_embds, 
                                             comment_mask, masked_token_ids,
                                             True, config)

params = to_immutable_dict( {'comments_encoder' : featurizer_params, 
                             'mlm_predictor' : ExtendedEncoder_params } )

In [ ]:
pure_logits_fn.apply(params['mlm_predictor'], 
                     subkey, comment_embds, 
                     comment_mask, masked_token_ids, 
                     True, config).shape

In [ ]:
@partial(jax.jit, static_argnums=(3,4))
def pure_featurizer(params, key, token_ids, training, config):
    key, subkey = jax.random.split(key)
    comment_embds = pure_featurizer_fn.apply(params, subkey, comment_ids, True, config)
    return comment_embds

@partial(jax.jit, static_argnums=(5,6))
def pure_logits(params, key, comment_embds, comment_mask, masked_token_ids, training, config):
    key, subkey = jax.random.split(key)
    logits = pure_logits_fn.apply(params, subkey, comment_embds, comment_mask, masked_token_ids, training=training, config=config)
    return logits


## Running Model and Getting Loss

In [ ]:
def cross_entropy(config, original_batch, logits, masked_token_ids):
    logits_mask = (masked_token_ids==config['mask_id'])
    logits = jax.vmap(jnp.multiply, (None,2), 2)(logits_mask,logits)
    labels = hk.one_hot(original_batch, config['vocab_size'])
    softmax_xent = -jnp.sum(labels*jax.nn.log_softmax(logits))
    total_masks = jnp.sum(logits_mask)
    if total_masks==0:
        return jnp.zeros(())
    softmax_xent /= total_masks
    return softmax_xent

In [ ]:
def loss(params, key, tree, config):
    """
    Calculates loss for all nodes of a single tree.
    The masked tokens of each location in a comment are predicted 
    conditioned on the embeddings of all the parent comments.
    """
    
    loss = 0

    #Prepare embeddings of each comment
    empty_elem = jnp.asarray([config['pad_id']]*config['max_length'], dtype=jnp.int16)
    batches = tree_to_batch(tree, config['featurizer_batch_size'],
                            key='tokenized_inputs', empty_elem=empty_elem, 
                            dtype=jnp.int16)
    #print(len(batches))
    encodings = []
    for batch in batches:
        key, subkey = jax.random.split(key)
        features = pure_featurizer(params['comments_encoder'], subkey, 
                                            batch, True, config)
        encodings.append(features)
    tree = batch_to_tree(tree, encodings, config['featurizer_batch_size'], 
                         key='comment_embds')

    #Calculate loss for each masked position in each comment.
    comment_batches = tree_to_batch(tree, config['mlm_batch_size'], key=None, 
                                    empty_elem={}, include_root=False)
    
    batches = tree_to_batch(tree, config['mlm_batch_size'],
                            key='tokenized_inputs', empty_elem=empty_elem,
                            include_root=False, dtype=jnp.int16)
    
    empty_elem = jnp.asarray([0]*config['d_model'], dtype=jnp.int16)
    
    for original_batch, comment_batch in zip(batches, comment_batches):
        parent_comment_embds, mask_for_embds = gather_batch_parents(tree, comment_batch, 
                                                                    config['max_length'], key='comment_embds', 
                                                                    empty_elem=empty_elem)
        key, subkey = jax.random.split(key)
        masked_batch, original_batch = mask_batch_mlm(subkey, config, original_batch)

        key, subkey = jax.random.split(key)
        logits = pure_logits(params['mlm_predictor'], subkey, parent_comment_embds, 
                             mask_for_embds, masked_batch, True, config)
        
        loss += cross_entropy(config, original_batch, logits, masked_batch)
    
    return loss

## Optimizer

In [ ]:
opt = get_adam_opt(config)
opt_state = opt.init(params)

In [ ]:
def update(opt_state, params, key, tree, config):
    batch_loss, grad = jax.value_and_grad(loss)(params, key, tree, config)
    updates, opt_state = opt.update(grad, opt_state)
    new_params = optax.apply_updates(params, updates)
    return new_params, opt_state, batch_loss

## Training Loop

In [ ]:
losses = []
for step, tree in enumerate(data_loader.tree_generator()):
    if step%100==0:
        print(f'[Step {step}]')
    
    tree = lm_tokeniser.tokenize_tree(tree)
    
    key, subkey = jax.random.split(key)
    params, opt_state, batch_loss = update(opt_state, params, subkey,
                                           tree, config)
    print(batch)
    losses.append(batch_loss)

    if step%100==0 and step!=0:
        print(sum(losses)/100)
        losses = []

[Step 0]
